This generates MBES2.0 with updated h2m 1.0.6.  
Kexin Dong  
Jan 15, 2023  

In [41]:
import os
import json
import requests
import gffutils
import Bio.Align
from Bio import pairwise2
from Bio.Data import CodonTable
import pegg
from itertools import product
import zipfile
import h2m
import pandas as pd
import gc

In [ ]:
# load refgenome
path_h_ref, path_m_ref = '/Users/gorkordkx/Documents/Database/RefGenome/ncbi-2023-09-12/GCF_000001405.25_GRCh37.p13_genomic.fna.gz', '/Users/gorkordkx/Documents/Database/RefGenome/mouse-2023-09-13/GCF_000001635.27_GRCm39_genomic.fna.gz'
# remember to replace the paths with yours
records_h, index_list_h = h2m.genome_loader(path_h_ref)
records_m, index_list_m  = h2m.genome_loader(path_m_ref)

In [ ]:
path_h_anno, path_m_anno = '/Users/gorkordkx/Documents/Database/Genecode/gencode_v19_GRCh37.db', '/Users/gorkordkx/Documents/Database/Genecode/gencode_vm33_GRCm39.db'
# remember to replace the paths with yours
db_h, db_m = h2m.anno_loader(path_h_anno), h2m.anno_loader(path_m_anno)

In [1]:
import pandas as pd
df_mutation = pd.read_csv('/Users/gorkordkx/Desktop/Flab - Drylab/Database/AACR-GENIE/v13.1/data_mutations_extended.txt', header=0, sep='\t', comment="#", na_values = 'Not Applicable')
# only keep MSK data
df = df_mutation[df_mutation['Center']=='MSK']
len(pd.DataFrame(df['Tumor_Sample_Barcode'].value_counts()))
df_mutation = df_mutation.iloc[:,[0, 37,2, 5,6,9,10,11,13,16]].reset_index(drop=True)
df_mutation = df_mutation[df_mutation['Center']=='MSK'].reset_index(drop=True)
df_mutation = df_mutation[df_mutation['Variant_Type'] == 'SNP'].reset_index(drop=True)
df_mutation = df_mutation.dropna(subset='HGVSc').reset_index(drop=True)
df_mutation['HGVSc'] = [x.split(':')[0] for x in df_mutation['HGVSc']]
df_mutation = df_mutation.drop_duplicates().reset_index(drop=True)

/var/folders/r5/h0l2l6hx09d0shb15xt6sdr40000gn/T/ipykernel_98233/3555049352.py:2: DtypeWarning: Columns (4,12,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mutation = pd.read_csv('/Users/gorkordkx/Desktop/Flab - Drylab/Database/AACR-GENIE/v13.1/data_mutations_extended.txt', header=0, sep='\t', comment="#", na_values = 'Not Applicable')


In [ ]:
df_mutation

,Hugo_Symbol,HGVSc,Center,Start_Position,End_Position,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,Tumor_Sample_Barcode
0,SH2D1A,ENST00000371139.4,MSK,123504048,123504048,Missense_Mutation,SNP,G,T,GENIE-MSK-P-0048298-T02-IM6
1,KRAS,ENST00000256078.4,MSK,25378562,25378562,Missense_Mutation,SNP,C,T,GENIE-MSK-P-0030423-T01-IH3
2,TMPRSS2,ENST00000332149.5,MSK,42866465,42866465,Missense_Mutation,SNP,C,T,GENIE-MSK-P-0052339-T02-IM6
3,TAP2,ENST00000374897.2,MSK,32800190,32800190,Nonsense_Mutation,SNP,G,A,GENIE-MSK-P-0002113-T02-IM6
4,INPP4B,ENST00000262992.4,MSK,143003288,143003288,Missense_Mutation,SNP,T,G,GENIE-MSK-P-0043596-T01-IM6
...,...,...,...,...,...,...,...,...,...,...
442234,RNF43,ENST00000407977.2,MSK,56432308,56432308,Missense_Mutation,SNP,A,G,GENIE-MSK-P-0046563-T01-IM6
442235,CDK8,ENST00000381527.3,MSK,26959350,26959350,Missense_Mutation,SNP,G,C,GENIE-MSK-P-0029742-T01-IM6
442236,TP53,ENST00000269305.4,MSK,7578404,7578404,Missense_Mutation,SNP,A,T,GENIE-MSK-P-0041476-T01-IM6
442237,EP300,ENST00000263253.7,MSK,41553365,41553365,Nonsense_Mutation,SNP,G,T,GENIE-MSK-P-0066956-T02-IM7


In [ ]:
df_mutation_count = df_mutation.groupby([df_mutation.columns.tolist()[x] for x in [0,1,3,4,5,6,7,8]], as_index = False).size()
df_mutation_count_sorted = df_mutation_count.sort_values(by = 'size')
df_mutation_count_sorted.columns =['gene_name_h', 'tx_id_h','start_h','end_h','class_h','type_h','ref_seq_h','alt_seq_h','count']
#df_mutation_count_sorted.reset_index(drop=True).to_csv('aacr_all_count.csv', index = False)

In [ ]:
df_mutation_count_sorted_cut = df_mutation_count_sorted[df_mutation_count_sorted['count']>2].reset_index(drop=True)
len(df_mutation_count_sorted_cut)

25291

In [ ]:
df_mutation_BE = df_mutation_count_sorted_cut.sort_values(by='gene_name_h').reset_index(drop=True)
df_mutation_BE['index'] = range(len(df_mutation_BE))
list_of_counts = df_mutation_BE['gene_name_h'].value_counts(sort=False).to_list()

In [ ]:
df_mutation_BE = df_mutation_BE.iloc[:,[0,1,2,3,5,6,7,4,8,9]]

In [ ]:
df_mutation_BE['class_h'].value_counts()

Missense_Mutation         20298
Nonsense_Mutation          3973
Splice_Site                 885
Intron                       49
Translation_Start_Site       42
5'UTR                        13
Nonstop_Mutation             11
3'UTR                         8
Splice_Region                 8
Silent                        4
Name: class_h, dtype: int64

In [ ]:
list_of_coding = [
    'Missense_Mutation',
    'Nonsense_Mutation',
    'Nonstop_Mutation',
    'Silent'
]
df_mutation_BE_coding = df_mutation_BE[[x in list_of_coding for x in df_mutation_BE['class_h']]].reset_index(drop=True)
df_mutation_BE_non_coding = df_mutation_BE[[x not in list_of_coding for x in df_mutation_BE['class_h']]].reset_index(drop=True)

# For non-coding ones

In [ ]:
len(list_of_counts)

568

In [ ]:
# df_mutation_BE_coding.to_csv('input_MSK_over_2_coding.csv', index=False)
# df_mutation_BE_non_coding.to_csv('input_MSK_over_2_non_coding.csv', index=False)

df_mutation_BE_non_coding = pd.read_csv('input_MSK_over_2_non_coding.csv')
list_of_counts = df_mutation_BE_non_coding['gene_name_h'].value_counts(sort=False).to_list()
df_mutation_BE = df_mutation_BE_non_coding

In [ ]:
result_list = []
df_list = []
t = 0
r = 0
for c_index, c_count in enumerate(list_of_counts):
    print(c_index)
    gene_name_h, tx_id_h  = df_mutation_BE.iloc[r,:2].to_list()
    # if gene_name_h in dict
    if gene_name_h in h2m.mouse_tx_id_dict.keys():
        tx_id_m = h2m.mouse_tx_id_dict[gene_name_h]
    else:
        try:
            gene_name_m = h2m.query(gene_name_h, show=False)[0]['gene_name_m']
            tx_id_m = h2m.get_tx_id(gene_name_m,'m', show=False)[3]
            # or input by yourself
        except:
            r = r+c_count
            continue
    
    for i in range(c_count):
        start_h, end_h, ty_h, ref_seq_h, alt_seq_h = df_mutation_BE.iloc[r,2:7].to_list()
        unique_index = df_mutation_BE.iloc[r,9]
        try:
            result = h2m.model(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, start_h, end_h, alt_seq_h, ty_h, ver = 37, ref_seq_h=ref_seq_h, param='BE', batch=False)
        except:
            r = r+1
        else:
            if type(result) == list: # multiple alternative modeling
                for k in result:
                    k['index'] = unique_index
                    r = r+1
                    result_list.append(k)
            else:
                result['index'] = unique_index
                r = r+1
                result_list.append(result)
        
    df_list.append(pd.DataFrame(result_list))
    result_list = []
    gc.collect()

df_result = pd.concat(df_list, ignore_index=True)
df_result.to_csv('non_coding_msk_over_2_complete_v2.csv',index=False)
df_output_noncoding = df_result[df_result['status']==True].reset_index(drop=True)
df_output_noncoding.to_csv('msk_modelable_non_coding_over_2_v2.csv', index=False)

# For coding ones
# df_mutation_BE_coding.to_csv('input_MSK_over_2_coding.csv', index=False)
# df_mutation_BE_non_coding.to_csv('input_MSK_over_2_non_coding.csv', index=False)

df_mutation_BE = pd.read_csv('input_MSK_over_2_coding.csv')
list_of_counts = df_mutation_BE['gene_name_h'].value_counts(sort=False).to_list()
result_list = []
df_list = []
t = 0
r = 0
for c_index, c_count in enumerate(list_of_counts):
    print(c_index)
    gene_name_h, tx_id_h  = df_mutation_BE.iloc[r,:2].to_list()

    # if gene_name_h in dict
    ##################################### get tx_id_m #############################
    if gene_name_h in h2m.mouse_tx_id_dict.keys():
        tx_id_m = h2m.mouse_tx_id_dict[gene_name_h]
    else:
        try:
            gene_name_m = h2m.query(gene_name_h, show=False)[0]['gene_name_m']
            tx_id_m = h2m.get_tx_id(gene_name_m,'m', show=False)[3]
            # or input by yourself
        except:
            r = r+c_count
            continue
    #################################### align ###################################
    align_input = h2m.get_tx_and_align(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, ver = 37, align = True)
    #################################### model ###################################
    for i in range(c_count):
        start_h, end_h, ty_h, ref_seq_h, alt_seq_h = df_mutation_BE.iloc[r,2:7].to_list()
        unique_index = df_mutation_BE.iloc[r,9]
        try:
            result = h2m.model(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, start_h, end_h, alt_seq_h, ty_h, ver = 37, ref_seq_h=ref_seq_h, param='BE', batch=True, align_input=align_input)
        except:
            r = r+1
        else:
            if type(result) == list: # multiple alternative modeling
                for k in result:
                    k['index'] = unique_index
                    r = r+1
                    result_list.append(k)
            else:
                result['index'] = unique_index
                r = r+1
                result_list.append(result)
    
    df_list.append(pd.DataFrame(result_list))
    result_list = []
    gc.collect()

df_result_2 = pd.concat(df_list, ignore_index=True)
df_result_2.to_csv('coding_msk_over_2_complete_v2.csv',index=False)


In [ ]:
df_output_coding.to_csv('msk_modelable_coding_over_2_v2.csv', index=False)
df_result_2.to_csv('coding_msk_over_2_complete_v2.csv',index=False)
df_output_coding = df_result_2[df_result_2['status']==True].reset_index(drop=True)
df_output_coding.to_csv('msk_modelable_coding_over_2_v2.csv', index=False)

df_result_bind = pd.concat([df_output_coding, df_output_noncoding], ignore_index=True)
df_result_bind = df_result_bind.sort_values(by='index').reset_index(drop=True)
df_result_bind.to_csv('msk_over_2_complete_v2.csv',index=False)

In [ ]:
df_result_bind = pd.read_csv('msk_over_2_complete_v2.csv')

In [ ]:
df_result_bind_list_index = [e for e,x in enumerate(list(df_result_bind['HGVSc_m'])) if x.split('>')[-1]=='None' ]
df_result_bind_wrong = df_result_bind[df_result_bind.index.isin(df_result_bind_list_index)]
df_result_bind_correct = df_result_bind[~df_result_bind.index.isin(df_result_bind_list_index)]
df_result_bind_wrong = df_result_bind_wrong.reset_index(drop=True)
df_result_bind_wrong['HGVSc_m'] = [x[:-9] for x in df_result_bind_wrong['HGVSc_m']]
df_result_bind_wrong['HGVSc_h'][0][-3:]
df_result_bind_wrong['HGVSc_m'] = [x+y[-3:] for x,y in zip(df_result_bind_wrong['HGVSc_m'],df_result_bind_wrong['HGVSc_h'])]


In [ ]:
df_mutation_BE_coding = pd.read_csv('input_MSK_over_2_coding.csv')
df_mutation_BE_non_coding = pd.read_csv('input_MSK_over_2_non_coding.csv')

In [ ]:
list(df_mutation.columns).index('HGVSp_Short')
df_mutation = df_mutation.iloc[:,[0, 37,2,5,6,9,10,11,13,16, 39]].reset_index(drop=True)
df_mutation = df_mutation[df_mutation['Center']=='MSK'].reset_index(drop=True)
df_mutation = df_mutation[df_mutation['Variant_Type'] == 'SNP'].reset_index(drop=True)
#df_mutation = df_mutation.dropna(subset='HGVSc').reset_index(drop=True)
#df_mutation['HGVSc'] = [x.split(':')[0] for x in df_mutation['HGVSc']]
df_mutation = df_mutation.drop_duplicates().reset_index(drop=True)

In [ ]:
df_result_bind_wrong = df_result_bind_wrong.drop('HGVSp_h', axis=1)

In [ ]:
df_mutation_to_merge = df_mutation.iloc[:,[0,3,4,7,8,10]]
df_mutation_to_merge['HGVSp_Short'] = [str(x) for x in df_mutation_to_merge['HGVSp_Short']]
df_mutation_to_merge['HGVSp_Short'] = [x[2:] for x in df_mutation_to_merge['HGVSp_Short']]
df_mutation_to_merge.columns = ['gene_name_h','start_h','end_h','ref_seq_h','alt_seq_h','HGVSp_h']
df_mutation_BE_new = pd.merge(df_mutation_BE_non_coding, df_mutation_to_merge, on =['gene_name_h','start_h','end_h','ref_seq_h','alt_seq_h'],how = 'left')
df_mutation_BE_new = df_mutation_BE_new.drop_duplicates().reset_index(drop=True)

In [ ]:
df_result_bind_wrong.columns

Index(['gene_name_h', 'gene_id_h', 'tx_id_m', 'chr_h', 'exon_num_h',
       'strand_h', 'start_h', 'end_h', 'ref_seq_h', 'alt_seq_h', 'HGVSc_h',
       'classification_h', 'exon_h', 'type_h', 'status', 'error', 'statement',
       'gene_name_m', 'gene_id_m', 'chr_m', 'exon_num_m', 'strand_m', 'type_m',
       'classification_m', 'exon_m', 'start_m_ori', 'end_m_ori',
       'ref_seq_m_ori', 'alt_seq_m_ori', 'HGVSc_m_ori', 'HGVSp_m_ori',
       'start_m', 'end_m', 'ref_seq_m', 'alt_seq_m', 'HGVSc_m', 'HGVSp_m',
       'index'],
      dtype='object')

In [ ]:
df_result_bind_wrong = pd.merge(df_result_bind_wrong, df_mutation_BE_new.iloc[:,[8,9,10]], on='index', how='left')

In [ ]:
df_result_bind_wrong_new = df_result_bind_wrong.iloc[:,list(range(11))+[38]+list(range(12,38))+[11]]

In [ ]:
df_mutation_BE_coding_for_bind = df_mutation_BE_coding.iloc[:,[8,9]]
df_result_bind_correct_new = pd.merge(df_result_bind_correct, df_mutation_BE_coding_for_bind, on='index', how='left')

In [ ]:
df_result_bind_correct_new = df_result_bind_correct_new.drop(columns=['start_m_ori',
       'end_m_ori', 'ref_seq_m_ori', 'alt_seq_m_ori', 'HGVSc_m_ori',
       'HGVSp_m_ori'])

In [ ]:
df_result_bind_wrong_new = df_result_bind_wrong.drop(columns=['start_m_ori',
       'end_m_ori', 'ref_seq_m_ori', 'alt_seq_m_ori', 'HGVSc_m_ori',
       'HGVSp_m_ori'])

In [ ]:
col_correct = list(df_result_bind_correct_new.columns)
col_wrong = list(df_result_bind_wrong_new.columns)
idx = [col_wrong.index(x) for x in col_correct]
df_result_bind_wrong_new = df_result_bind_wrong_new.iloc[:,idx]

In [43]:
df_result_bind_new = pd.concat([df_result_bind_wrong_new, df_result_bind_correct_new])
df_result_bind_new = df_result_bind_new.sort_values(by='index').reset_index(drop=True)
#col_index = list(range(26))+list(range(26+6, len(df_result_bind_new.columns)))
#df_result_bind_new = df_result_bind_new.iloc[:,col_index]

In [45]:
df_result_bind_new.to_csv('msk_BE_modelable_pts_over_2.csv', index=False)

In [ ]:
# result_list = []
# df_list = []
# t = 0
# r = 0
# for c_index, c_count in enumerate(list_of_counts):

#     print(c_index)
#     gene_name_h, tx_id_h  = df_mutation_BE.iloc[r,:2].to_list()
#     unique_index = df_mutation_BE.iloc[r,9]
#     # if gene_name_h in dict
#     if gene_name_h in h2m.mouse_tx_id_dict.keys():
#         tx_id_m = h2m.mouse_tx_id_dict[gene_name_h]

#         align_input = h2m.get_tx_and_align(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, ver = 37, align = True)
#         for i in range(c_count):
#             start_h, end_h, ty_h, ref_seq_h, alt_seq_h = df_mutation_BE.iloc[r,2:7].to_list()
#             try:
#                 result = h2m.model(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, start_h, end_h, alt_seq_h, ty_h, ver = 37, ref_seq_h=ref_seq_h, param='BE', batch=True, align_input = align_input)
#                 if type(result) == list: # multiple alternative modeling
#                     for k in result:
#                         k['index'] = unique_index
#                         r = r+1
#                         result_list.append(k)
#                 else:
#                     result['index'] = unique_index
#                     r = r+1
#                     result_list.append(result)
                    
#             except:
#                 r = r+1
#     else:
#         try:
#             gene_name_m = h2m.query(gene_name_h, show=False)[0]['gene_name_m']
#             tx_id_m = h2m.get_tx_id(gene_name_m,'m', show=False)[3]
#             # or input by yourself
#             align_input = h2m.get_tx_and_align(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, ver = 37, align = True)
#             for i in range(c_count):
#                 start_h, end_h, ty_h, ref_seq_h, alt_seq_h = df_mutation_BE.iloc[r,2:7].to_list()
#                 try:
#                     result = h2m.model(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, start_h, end_h, alt_seq_h, ty_h, ver = 37, ref_seq_h=ref_seq_h, param='BE', batch=True, align_input = align_input)
#                     if type(result) == list: # multiple alternative modeling
#                         for k in result:
#                             k['index'] = unique_index
#                             r = r+1
#                             result_list.append(k)
#                     else:
#                         result['index'] = unique_index
#                         r = r+1
#                         result_list.append(result)
                        
#                 except:
#                     r = r+1
#         except:
#             r = r+1

#     df_list.append(pd.DataFrame(result_list))
#     result_list = []
#     gc.collect()

# df_result = pd.concat(df_list, ignore_index=True)

In [19]:
h2m.Translate('CAG')

Seq('Q')

In [15]:
'ACT'

'ACT'

In [22]:
gene_region_anno_h = h2m.GetTx(records_h, index_list_h, db_h, 'ENST00000269305.4', 'h', 37, 300)[7]

In [46]:
gene_id_h, gene_name_h, chr_h, n_exon_h, gene_seq_h, gene_index_h, gene_region_anno_h, gene_region_idx_h = h2m.GetTx(records_h, index_list_h, db_h, 'ENST00000269305.4', 'h', 37, 300)
tx_id_m = h2m.get_tx_id('Trp53','m', show=False)[3]
gene_id_m, gene_name_m, chr_m, n_exon_m, gene_seq_m, gene_index_m, gene_region_anno_m, gene_region_idx_m = h2m.GetTx(records_m, index_list_m, db_m, tx_id_m, 'm', 37, 300)

In [189]:
start_h, end_h = 7577018, 7577018
splicing_size = 30

In [190]:
strand_m = '-'
strand_h = '+'

In [ ]:
def GateFlankH(start_h, end_h, len_seq_h, align_idx_h, flank_size):
    global status, error, statement
    flank_left = min([start_h, flank_size])
    flank_right = min([len_seq_h-end_h-1, flank_size])
    s_flank, e_flank = start_h-flank_left, end_h+flank_right
    identity = set(list(range(s_flank,e_flank+1))).issubset(set(align_idx_h))
    if identity == False:
        status = False
        statement = 'Error 4: Flanked segments are not identical.'
        error = 4

In [28]:
x = pairwise2.align.localxx('AATATA', 'AATATATATCCGGGCCCCCGCGCGC')
h2m.match_index(*x[0])

[0, 1, 2, 3, 4, 7]

In [ ]:
set_of_mut_idx = set([gene_region_idx_h[gene_index_h.index(x)] for x in range(start_h, end_h+1)]) 
set_of_mut_idx_idx_h = set(condition(gene_region_idx_h, set_of_mut_idx))

In [126]:
import json
file_name = '/Users/gorkordkx/Documents/GitHub/H2M-py/mouse_tx_id_dict.json'
with open(file_name, 'r') as file:
    mouse_tx_id_dict = json.load(file)

In [268]:
df = df[df['Variant_Classification']=='Splice_Site'].reset_index(drop=True)
df = df[df['Hugo_Symbol']=='TP53'].reset_index(drop=True)

In [ ]:
import module_14 as module_4
result_list = []
error_index_list = []
r = 0
t = 0
for c_index, c_count in enumerate([10]):
    print(c_index)
    gene_name_h, tx_id_h  = 'TP53', 'ENST00000269305.4'
    # if gene_name_h in dict
    if gene_name_h in mouse_tx_id_dict.keys():
        tx_id_m = mouse_tx_id_dict[gene_name_h]
    else:
        gene_name_m = module_4.query(gene_name_h, show=False)[0]['gene_name_m']
        tx_id_m = module_4.get_tx_id(gene_name_m,'m', show=False)[3]
        # or input by yourself

    align_input = module_4.get_tx_and_align(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, ver = 37, align = True)
    
    for i in range(c_count):
        start_h, end_h, ty_h, ref_seq_h, alt_seq_h = df.iloc[r,[5,6,10,11,13]].tolist()
        try:
            result = module_4.model(records_h, index_list_h, records_m, index_list_m, db_h, db_m, tx_id_h, tx_id_m, start_h, end_h, alt_seq_h, ty_h, ver = 37, ref_seq_h=ref_seq_h, param='BE', batch=True, align_input = align_input)
            if type(result) == list: # multiple alternative modeling
                for k in result:
                    k['index'] = t
                    t = t +1
                    r = r+1
                    result_list.append(k)
            else:
                result['index'] = t
                t = t + 1
                r = r+1
                result_list.append(result)
                
        except:
            error_index_list.append(r)
            r = r+1

    # pd.DataFrame(result_list).to_csv(csv_file_path, mode='a', header=False, index=False)
    # if len(error_index_list)>0:
    #     with open('error.txt', 'a') as file:  # 使用'a'模式以便于在文件末尾追加内容
    #         for item in error_index_list:
    #             file.write(str(item) + "\n")
    # error_index_list = []
    # result_list = []
    # gc.collect()

In [26]:
h2m.Translate('ATG')

Seq('M')